<a href="https://colab.research.google.com/github/s-c-soma/AdvanceDeeplearning-CMPE-297/blob/master/CA_baselineAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description


This colab is being used to prepare Calfornia baseline analysis data for all counties. We are prepapreing here two sets of data. 

* 1. CA_combined data from merging all the listed datasets- 
> * Merge Social Distancing, 
> * mobility data,
> * Covid Cases, 
> * Population Density 
> * Mask data for rule

* 2. Merging Social Distancing Inertia (Maryland data) and Covid data integration

In [119]:
!pip install wget


In [120]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [121]:
!ls  '/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/'

CA_combined_Sep09.csv
CA-CovidAug22.csv
CA_socialDistancingInertiaData_Sep09.csv
Maryland_University_SocialDistancing_Data
mask_rule.csv
Newyork_combinedAug22.csv
NY_sd_inertia_labeledData_Aug26.csv
NY_socialDistancingInertiaDataAug25.csv
Social_Distancing_Inertia_County.csv


# Load library

In [205]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import wget
from pandas import Series, datetime
from pandas.plotting import scatter_matrix, autocorrelation_plot
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
import random
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from xgboost import XGBClassifier
from sklearn.mixture import GaussianMixture
from datetime import datetime

In [123]:
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
import warnings; warnings.simplefilter('ignore')

# 1 Load Data Sets

## 1.1 County Population

In [124]:
US_population_filename = wget.download('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv')
county_population_US = pd.read_csv(US_population_filename,low_memory=False)
print(county_population_US.shape)

county_population_US.head(2)

(3195, 4)


,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869


In [125]:
CA_population = county_population_US[county_population_US['State'] == 'CA']
print(CA_population.shape)
CA_population.head(2)

(60, 4)


,countyFIPS,County Name,State,population
191,0,Statewide Unallocated,CA,0
192,6000,Grand Princess Cruise Ship,CA,0


In [126]:
CA_population['County Name'].unique()

array(['Statewide Unallocated', 'Grand Princess Cruise Ship',
       'Alameda County', 'Alpine County', 'Amador County', 'Butte County',
       'Calaveras County', 'Colusa County', 'Contra Costa County',
       'Del Norte County', 'El Dorado County', 'Fresno County',
       'Glenn County', 'Humboldt County', 'Imperial County',
       'Inyo County', 'Kern County', 'Kings County', 'Lake County',
       'Lassen County', 'Los Angeles County', 'Madera County',
       'Marin County', 'Mariposa County', 'Mendocino County',
       'Merced County', 'Modoc County', 'Mono County', 'Monterey County',
       'Napa County', 'Nevada County', 'Orange County', 'Placer County',
       'Plumas County', 'Riverside County', 'Sacramento County',
       'San Benito County', 'San Bernardino County', 'San Diego County',
       'San Francisco County', 'San Joaquin County',
       'San Luis Obispo County', 'San Mateo County',
       'Santa Barbara County', 'Santa Clara County', 'Santa Cruz County',
       'Shast

In [127]:
LA_county_population = CA_population[CA_population['County Name'] == 'Los Angeles County']
LA_county_population

,countyFIPS,County Name,State,population
211,6037,Los Angeles County,CA,10039107


## 1.2 Land Area 

In [128]:
landarea_filename = wget.download('https://raw.githubusercontent.com/ykzeng/covid-19/master/data/census-landarea-all.csv')
landarea_data = pd.read_csv(landarea_filename,low_memory=False)
print(landarea_data.shape)


(3195, 52)


In [129]:
landarea_data.head(2)

,fips,PST045212,PST040210,PST120212,POP010210,AGE135212,AGE295212,AGE775212,SEX255212,RHI125212,RHI225212,RHI325212,RHI425212,RHI525212,RHI625212,RHI725212,RHI825212,POP715211,POP645211,POP815211,EDU635211,EDU685211,VET605211,LFE305211,HSG010211,HSG445211,HSG096211,HSG495211,HSD410211,HSD310211,INC910211,INC110211,PVY020211,BZA010211,BZA110211,BZA115211,NES010211,SBO001207,SBO315207,SBO115207,SBO215207,SBO515207,SBO415207,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030212,LND110210,POP060210
0,0,313914040,308747508,1.7,308745538,6.4,23.5,13.7,50.8,77.9,13.1,1.2,5.1,0.2,2.4,16.9,63.0,84.6,12.8,20.3,85.4,28.2,22215303,25.4,132312404,66.1,25.9,186200,114761359,2.60,27915,52762,14.3,7354043,113425965,1.3,22491080,27092908,7.1,0.9,5.7,0.1,8.3,28.8,5319456312,4174286516,3917663456,12990,613795732,829658,3531905.43,87.4
1,1000,4822023,4779745,0.9,4779736,6.3,23.3,14.5,51.5,70.0,26.5,0.7,1.2,0.1,1.5,4.1,66.6,84.5,3.4,5.0,81.9,22.0,403982,24.0,2182088,70.7,15.5,120800,1831269,2.53,23483,42934,17.6,97743,1573138,0.3,321641,382350,14.8,0.8,1.8,0.1,1.2,28.1,112858843,52252752,57344851,12364,6426342,13506,50645.33,94.4


In [130]:
landarea_data['fips'].unique()

array([    0,  1000,  1001, ..., 56041, 56043, 56045])

In [131]:
#LND110210

landarea_data[landarea_data['fips'] == 36081]['LND110210']

1902    108.53
Name: LND110210, dtype: float64

#### Population density

In [132]:
density = 2253858/108.53

density

20767.142725513684

LA population density

In [133]:
LA_population_density=pd.merge(LA_county_population,landarea_data,how='inner', left_on=['countyFIPS'], right_on = ['fips'])

In [134]:
LA_population_density['population_density']= LA_population_density['population']/LA_population_density['LND110210']
LA_population_density = LA_population_density[['fips','County Name', 'State', 'population','LND110210','population_density']].copy()
LA_population_density.tail(3)

,fips,County Name,State,population,LND110210,population_density
0,6037,Los Angeles County,CA,10039107,4057.88,2473.978284


## 1.3 Mask data

In [135]:
mask_rule_data = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/mask_rule.csv',low_memory=False)
 
mask_rule_data.head()

,state_name,Law Date,Masks Required?,Type of Requirement,mask_required_rule,requirement_code
0,Louisiana,2020-07-13,Entire State,Everywhere in public where social distancing i...,0,3
1,Kentucky,2020-07-10,Entire State,Everywhere in public where social distancing i...,0,3
2,West Virginia,2020-07-07,Entire State,All Public Indoor Spaces (where social distanc...,0,1
3,Texas,2020-07-03,Entire State,Everywhere in public where social distancing i...,0,4
4,Kansas,2020-07-03,Entire State,Everywhere in public where social distancing i...,0,3


In [136]:
CA_mask_rule = mask_rule_data[mask_rule_data['state_name'] == 'California']
CA_mask_rule

,state_name,Law Date,Masks Required?,Type of Requirement,mask_required_rule,requirement_code
9,California,2020-06-18,Entire State,Everywhere in public where social distancing i...,0,3


In [137]:
mask_filename = wget.download('https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv')
mask_filename

'mask-use-by-county (6).csv'

In [138]:

mask_data = pd.read_csv(mask_filename,low_memory=False)
print(mask_data.shape)

(3142, 6)


In [139]:
mask_data.head()

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459


In [140]:
#los angeles
mask_data[mask_data['COUNTYFP'] == 6037]

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
204,6037,0.021,0.013,0.049,0.131,0.786


## 1.4 Mobility Data 

### Google mobility data

In [141]:
google_data_url = 'https://raw.githubusercontent.com/ActiveConclusion/COVID19_mobility/master/google_reports/mobility_report_US.csv'

google_filename = wget.download(google_data_url)
google_mobility_data = pd.read_csv(google_filename,low_memory=False)
google_mobility_data.head(2)

,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential
0,Total,Total,2020-02-15,6.0,2.0,15.0,3.0,2.0,-1.0
1,Total,Total,2020-02-16,7.0,1.0,16.0,2.0,0.0,-1.0


In [142]:
CA_GoogleMobility_data = google_mobility_data[google_mobility_data['state'] == 'California']
print(CA_GoogleMobility_data.shape)
CA_GoogleMobility_data.tail(2)

(11445, 9)


,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential
43405,California,Yuba County,2020-09-03,3.0,17.0,NaN,-18.0,-33.0,6.0
43406,California,Yuba County,2020-09-04,-5.0,15.0,NaN,-14.0,-35.0,7.0


### Apple mobility data

In [143]:
apple_report_url = 'https://raw.githubusercontent.com/ActiveConclusion/COVID19_mobility/master/apple_reports/apple_mobility_report_US.csv'

apple_filename = wget.download(apple_report_url)
apple_mobility_data = pd.read_csv(apple_filename,low_memory=False)
apple_mobility_data.tail(2)

,state,county_and_city,geo_type,date,driving,transit,walking
535564,Wyoming,Uinta County,county,2020-09-06,331.32,NaN,NaN
535565,Wyoming,Uinta County,county,2020-09-07,674.59,NaN,NaN


In [144]:
CA_AppleMobility_data = apple_mobility_data[apple_mobility_data['state'] == 'California']
print(CA_AppleMobility_data.shape)
CA_AppleMobility_data.tail(2)

(17839, 7)


,state,county_and_city,geo_type,date,driving,transit,walking
48456,California,Yuba County,county,2020-09-06,66.30,NaN,NaN
48457,California,Yuba County,county,2020-09-07,53.49,NaN,NaN


In [145]:
CA_AppleMobility_data['county'] = CA_AppleMobility_data['county_and_city']

In [146]:
CA_AppleMobility_data = CA_AppleMobility_data[['state','county','date','driving','transit','walking']]

CA_AppleMobility_data.tail(2)

,state,county,date,driving,transit,walking
48456,California,Yuba County,2020-09-06,66.30,NaN,NaN
48457,California,Yuba County,2020-09-07,53.49,NaN,NaN


In [147]:
CA_mobility_data = pd.merge(CA_GoogleMobility_data,CA_AppleMobility_data,how='outer' ,on=['state','county','date'])
CA_mobility_data.shape

(18366, 12)

In [148]:
CA_mobility_data = CA_mobility_data.fillna(0)
CA_mobility_data.tail()

,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking
18361,California,Yuba County,2020-02-13,0.0,0.0,0.0,0.0,0.0,0.0,17.75,0.0,0.0
18362,California,Yuba County,2020-02-14,0.0,0.0,0.0,0.0,0.0,0.0,40.51,0.0,0.0
18363,California,Yuba County,2020-09-05,0.0,0.0,0.0,0.0,0.0,0.0,94.16,0.0,0.0
18364,California,Yuba County,2020-09-06,0.0,0.0,0.0,0.0,0.0,0.0,66.30,0.0,0.0
18365,California,Yuba County,2020-09-07,0.0,0.0,0.0,0.0,0.0,0.0,53.49,0.0,0.0


### Los Angeles county mobility data

In [149]:
LA_mobility_data = CA_mobility_data[CA_mobility_data['county'] == 'Los Angeles']
print(LA_mobility_data.shape)
LA_mobility_data.tail()

(237, 12)


,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking
13948,California,Los Angeles,2020-09-03,0.0,0.0,0.0,0.0,0.0,0.0,14.65,-49.15,17.76
13949,California,Los Angeles,2020-09-04,0.0,0.0,0.0,0.0,0.0,0.0,38.07,-43.38,40.78
13950,California,Los Angeles,2020-09-05,0.0,0.0,0.0,0.0,0.0,0.0,23.47,-52.76,45.19
13951,California,Los Angeles,2020-09-06,0.0,0.0,0.0,0.0,0.0,0.0,2.19,-60.04,19.58
13952,California,Los Angeles,2020-09-07,0.0,0.0,0.0,0.0,0.0,0.0,-7.18,-57.82,2.91


In [150]:
LA_mobility_data.head()

,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking
13716,California,Los Angeles,2020-01-13,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
13717,California,Los Angeles,2020-01-14,0.0,0.0,0.0,0.0,0.0,0.0,5.93,3.78,5.72
13718,California,Los Angeles,2020-01-15,0.0,0.0,0.0,0.0,0.0,0.0,12.23,5.28,11.35
13719,California,Los Angeles,2020-01-16,0.0,0.0,0.0,0.0,0.0,0.0,11.73,3.12,11.68
13720,California,Los Angeles,2020-01-17,0.0,0.0,0.0,0.0,0.0,0.0,30.20,9.34,34.45


## 1.5 Social Distancing metric 

In [151]:
socialdistancing_url = 'https://raw.githubusercontent.com/descarteslabs/DL-COVID-19/master/DL-us-mobility-daterow.csv'

socialdistancing_file = wget.download(socialdistancing_url)
socialdistancing_data = pd.read_csv(socialdistancing_file,low_memory=False)
socialdistancing_data.tail(2)



,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
508644,2020-09-05,US,2,Wyoming,Uinta County,56041.0,754,11.328,361
508645,2020-09-06,US,2,Wyoming,Uinta County,56041.0,791,5.016,160


In [152]:
newyork_socialdistancing_data= socialdistancing_data[(socialdistancing_data['admin1']=='California') &
                                           (socialdistancing_data['admin_level']== 2)]

newyork_socialdistancing_data.head()

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
31068,2020-03-01,US,2,California,Alameda County,6001.0,16737,3.684,58
31069,2020-03-02,US,2,California,Alameda County,6001.0,17269,6.182,98
31070,2020-03-03,US,2,California,Alameda County,6001.0,18181,6.271,100
31071,2020-03-04,US,2,California,Alameda County,6001.0,18495,6.209,99
31072,2020-03-05,US,2,California,Alameda County,6001.0,17496,6.654,106


In [153]:
LA_social_distancing = newyork_socialdistancing_data[newyork_socialdistancing_data['admin2'] == 'Los Angeles County']
print(LA_social_distancing.shape)
LA_social_distancing.head()

(188, 9)


,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
34264,2020-03-01,US,2,California,Los Angeles County,6037.0,110713,3.996,60
34265,2020-03-02,US,2,California,Los Angeles County,6037.0,112327,6.502,98
34266,2020-03-03,US,2,California,Los Angeles County,6037.0,116225,6.591,99
34267,2020-03-04,US,2,California,Los Angeles County,6037.0,118658,6.595,100
34268,2020-03-05,US,2,California,Los Angeles County,6037.0,113023,7.008,106


# 2 Load Social Distancing Inertia Data [Maryland University]

In [154]:
maryland_sd_inertia = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/Social_Distancing_Inertia_County.csv',low_memory=False)
maryland_sd_inertia.head()

,CTFIPS,CTNAME,STFIPS,Social distancing index,% staying home,Trips/person,% out-of-county trips,% out-of-state trips,Miles/person,Work trips/person,Non-work trips/person,New COVID cases,Population,% change in consumption,date,Transit mode share,% people older than 60,Median income,% African Americans,% Hispanic Americans,% Male,Population density,Employment density,# hot spots/1000 people,Hospital beds/1000 people,ICUs/1000 people,# contact tracing workers/1000 people,# contact tracing workers/1000 people rank,% people older than 60 rank,COVID exposure/1000 people,#days: decreasing ILI cases,Unemployment claims/1000 people,Unemployment rate,% working from home,Cumulative inflation rate,COVID death rate,New cases/1000 people,Active cases/1000 people,#days: decreasing COVID cases,% hospital bed utilization,Testing capacity,Tests done/1000 people,% ICU utilization,Ventilator shortage,Imported COVID cases
0,1001,Autauga County,1,54,27,2.25,38.8,1.2,24.7,0.25,2.00,0,55601,-22.5,1/1/2020,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.3,0.0,0.0,0.0,0,47.1,0.0,0.0,0.0,0,0
1,1001,Autauga County,1,25,18,2.83,43.5,0.9,37.6,0.62,2.21,0,55601,-14.3,1/2/2020,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.3,0.0,0.0,0.0,0,47.1,0.0,0.0,0.0,0,0
2,1001,Autauga County,1,15,15,3.18,42.2,1.0,40.3,0.62,2.57,0,55601,-0.4,1/3/2020,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.3,0.0,0.0,0.0,0,47.1,0.0,0.0,0.0,0,0
3,1001,Autauga County,1,33,20,2.85,43.9,1.2,29.5,0.33,2.52,0,55601,-2.3,1/4/2020,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.3,0.0,0.0,0.0,0,47.1,0.0,0.0,0.0,0,0
4,1001,Autauga County,1,40,22,2.56,42.8,1.2,30.9,0.23,2.33,0,55601,-9.7,1/5/2020,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.3,0.0,0.0,0.0,0,47.1,0.0,0.0,0.0,0,0


## Fixdate column of Social Distancing inertia

In [155]:
def fixDate(x, appendTwenty):
  arr = x.split('/')
  m = arr[0]
  d = arr[1]
  y = arr[2]

  if int(m) < 10:
    m = '0'+str(m)
  if int(d) < 10:
    d = '0'+str(d)
  if appendTwenty == True:
    return '20'+y+'-'+m+'-'+d
  else:
      return y+'-'+m+'-'+d

In [156]:
maryland_sd_inertia['date'] = maryland_sd_inertia['date'].apply(lambda x: fixDate(x,False))
maryland_sd_inertia.reset_index(drop=True)

,CTFIPS,CTNAME,STFIPS,Social distancing index,% staying home,Trips/person,% out-of-county trips,% out-of-state trips,Miles/person,Work trips/person,Non-work trips/person,New COVID cases,Population,% change in consumption,date,Transit mode share,% people older than 60,Median income,% African Americans,% Hispanic Americans,% Male,Population density,Employment density,# hot spots/1000 people,Hospital beds/1000 people,ICUs/1000 people,# contact tracing workers/1000 people,# contact tracing workers/1000 people rank,% people older than 60 rank,COVID exposure/1000 people,#days: decreasing ILI cases,Unemployment claims/1000 people,Unemployment rate,% working from home,Cumulative inflation rate,COVID death rate,New cases/1000 people,Active cases/1000 people,#days: decreasing COVID cases,% hospital bed utilization,Testing capacity,Tests done/1000 people,% ICU utilization,Ventilator shortage,Imported COVID cases
0,1001,Autauga County,1,54,27,2.25,38.8,1.2,24.7,0.25,2.00,0,55601,-22.5,2020-01-01,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.30,0.0,0.0000,0.00,0,47.100000,0.0,0.00,0.00,0,0
1,1001,Autauga County,1,25,18,2.83,43.5,0.9,37.6,0.62,2.21,0,55601,-14.3,2020-01-02,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.30,0.0,0.0000,0.00,0,47.100000,0.0,0.00,0.00,0,0
2,1001,Autauga County,1,15,15,3.18,42.2,1.0,40.3,0.62,2.57,0,55601,-0.4,2020-01-03,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.30,0.0,0.0000,0.00,0,47.100000,0.0,0.00,0.00,0,0
3,1001,Autauga County,1,33,20,2.85,43.9,1.2,29.5,0.33,2.52,0,55601,-2.3,2020-01-04,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.30,0.0,0.0000,0.00,0,47.100000,0.0,0.00,0.00,0,0
4,1001,Autauga County,1,40,22,2.56,42.8,1.2,30.9,0.23,2.33,0,55601,-9.7,2020-01-05,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.30,0.0,0.0000,0.00,0,47.100000,0.0,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732081,56037,Sweetwater County,56,19,22,4.62,25.1,10.5,63.0,0.58,4.04,0,43051,37.8,2020-08-20,2.52,17,73008,0.8,16.0,51.87,4,3,134,3.76,0.50,0.017,47.0,6,17.1,105,3.8,12.6,26.2,1.51,0.3,0.0542,2.19,0,37.716682,5.4,111.08,1.31,3,90
732082,56039,Teton County,56,33,21,4.12,33.0,20.7,30.2,0.39,3.73,4,23081,0.8,2020-08-20,4.05,20,83831,1.1,14.9,51.65,5,7,297,3.76,0.50,0.017,47.0,16,17.1,105,3.8,12.6,26.2,1.51,0.3,0.1155,2.19,0,37.716682,5.4,111.08,1.31,3,100
732083,56041,Uinta County,56,21,20,4.44,36.8,24.5,79.0,0.65,3.79,0,20299,52.1,2020-08-20,3.60,18,58235,0.1,9.1,50.97,10,5,147,3.76,0.50,0.017,47.0,9,17.1,105,3.8,12.6,26.2,1.51,0.3,0.0985,2.19,14,37.716682,5.4,111.08,1.31,3,80
732084,56043,Washakie County,56,35,41,3.34,19.0,2.1,59.4,0.85,2.50,2,7885,20.9,2020-08-20,0.03,28,53426,0.0,14.2,50.89,4,3,205,3.76,0.50,0.017,47.0,73,17.1,105,3.8,12.6,26.2,1.51,0.3,0.2536,2.19,5,37.716682,5.4,111.08,1.31,3,7


# 3.Combining CA -all county data 

In [157]:
CA_covidcases = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/CA-CovidAug22.csv', low_memory =False)
CA_covidcases.tail()

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
12562,2020-08-17,California,90006.0,0,0,0,0
12563,2020-08-18,California,90006.0,0,0,0,0
12564,2020-08-19,California,90006.0,0,0,0,0
12565,2020-08-20,California,90006.0,0,0,0,0
12566,2020-08-21,California,90006.0,0,0,0,0


In [158]:
#CA_covidcases['Date'] = CA_covidcases['Date'].apply(lambda x: fixDate(x))
CA_covidcases.reset_index(drop=True)
CA_covidcases.head(2)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
0,2020-01-22,California,6001.0,0,0,0,0
1,2020-01-23,California,6001.0,0,0,0,0


In [159]:
CA_covidcases['FIPS'] = CA_covidcases['FIPS'].apply(lambda x :int(x))

In [160]:
CA_covidcases.head(2)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
0,2020-01-22,California,6001,0,0,0,0
1,2020-01-23,California,6001,0,0,0,0


In [161]:
CA_covidcases.shape

(12567, 7)

## Combining mask data -CA county

In [162]:
CA_covidcases['mask_rule_active'] = CA_covidcases['Date'].apply(lambda x : 1 if x >= CA_mask_rule['Law Date'].values[0] else 0 )


In [163]:
#Mask
CA_mask = pd.merge(CA_covidcases,mask_data,how='inner',left_on=['FIPS'],right_on=['COUNTYFP'])
CA_mask.head(2)


,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,2020-01-22,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795
1,2020-01-23,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795


In [164]:
CA_mask.shape

(12354, 14)

In [165]:
def updateMaskWearingPercent(x):
  #print(x)
  if x.mask_rule_active == 1:
    return x.ALWAYS
  else:
    return 0.0

In [166]:
CA_mask['mask_wearing_percent'] = CA_mask.apply(lambda x : updateMaskWearingPercent(x),axis=1)
 
CA_mask.tail()

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,mask_wearing_percent
12349,2020-08-17,California,6115,758,4,0,0,1,6115,0.057,0.035,0.071,0.167,0.669,0.669
12350,2020-08-18,California,6115,786,4,28,0,1,6115,0.057,0.035,0.071,0.167,0.669,0.669
12351,2020-08-19,California,6115,821,4,35,0,1,6115,0.057,0.035,0.071,0.167,0.669,0.669
12352,2020-08-20,California,6115,821,4,0,0,1,6115,0.057,0.035,0.071,0.167,0.669,0.669
12353,2020-08-21,California,6115,829,4,8,0,1,6115,0.057,0.035,0.071,0.167,0.669,0.669


In [167]:
CA_mask.head()

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,mask_wearing_percent
0,2020-01-22,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0
1,2020-01-23,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0
2,2020-01-24,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0
3,2020-01-25,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0
4,2020-01-26,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0


## Combining population density data

In [168]:
# Pop density
#landarea_data[landarea_data['fips'] == 36081]['LND110210']

CA_landarea_df = pd.merge(CA_mask,landarea_data,how='inner',left_on=['FIPS'],right_on=['fips'])
CA_landarea_df.head(2)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,mask_wearing_percent,fips,PST045212,PST040210,PST120212,POP010210,AGE135212,AGE295212,AGE775212,SEX255212,RHI125212,RHI225212,RHI325212,RHI425212,RHI525212,RHI625212,RHI725212,RHI825212,POP715211,POP645211,POP815211,EDU635211,EDU685211,VET605211,LFE305211,HSG010211,HSG445211,HSG096211,HSG495211,HSD410211,HSD310211,INC910211,INC110211,PVY020211,BZA010211,BZA110211,BZA115211,NES010211,SBO001207,SBO315207,SBO115207,SBO215207,SBO515207,SBO415207,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030212,LND110210,POP060210
0,2020-01-22,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0,6001,1554720,1510271,2.9,1510271,6.3,22.1,11.8,51.0,52.3,12.8,1.2,27.6,1.0,5.1,22.7,33.5,84.0,30.7,42.8,86.0,40.8,68414,28.1,584622,54.5,37.9,558300,536160,2.73,34937,70821,11.8,35921,567151,0.8,118341,140342,6.8,0.0,24.4,0.0,8.9,32.2,25236865,35674007,17909054,12359,2807766,2745,739.02,2043.6
1,2020-01-23,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0,6001,1554720,1510271,2.9,1510271,6.3,22.1,11.8,51.0,52.3,12.8,1.2,27.6,1.0,5.1,22.7,33.5,84.0,30.7,42.8,86.0,40.8,68414,28.1,584622,54.5,37.9,558300,536160,2.73,34937,70821,11.8,35921,567151,0.8,118341,140342,6.8,0.0,24.4,0.0,8.9,32.2,25236865,35674007,17909054,12359,2807766,2745,739.02,2043.6


In [169]:
CA_landarea_df.shape

(12354, 67)

In [170]:
CA_landarea = pd.merge(CA_landarea_df,CA_population,how='inner',left_on=['FIPS'],right_on=['countyFIPS'])

CA_landarea.head(2)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,mask_wearing_percent,fips,PST045212,PST040210,PST120212,POP010210,AGE135212,AGE295212,AGE775212,SEX255212,RHI125212,RHI225212,RHI325212,RHI425212,RHI525212,RHI625212,RHI725212,RHI825212,POP715211,POP645211,POP815211,EDU635211,EDU685211,VET605211,LFE305211,HSG010211,HSG445211,HSG096211,HSG495211,HSD410211,HSD310211,INC910211,INC110211,PVY020211,BZA010211,BZA110211,BZA115211,NES010211,SBO001207,SBO315207,SBO115207,SBO215207,SBO515207,SBO415207,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030212,LND110210,POP060210,countyFIPS,County Name,State,population
0,2020-01-22,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0,6001,1554720,1510271,2.9,1510271,6.3,22.1,11.8,51.0,52.3,12.8,1.2,27.6,1.0,5.1,22.7,33.5,84.0,30.7,42.8,86.0,40.8,68414,28.1,584622,54.5,37.9,558300,536160,2.73,34937,70821,11.8,35921,567151,0.8,118341,140342,6.8,0.0,24.4,0.0,8.9,32.2,25236865,35674007,17909054,12359,2807766,2745,739.02,2043.6,6001,Alameda County,CA,1671329
1,2020-01-23,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0,6001,1554720,1510271,2.9,1510271,6.3,22.1,11.8,51.0,52.3,12.8,1.2,27.6,1.0,5.1,22.7,33.5,84.0,30.7,42.8,86.0,40.8,68414,28.1,584622,54.5,37.9,558300,536160,2.73,34937,70821,11.8,35921,567151,0.8,118341,140342,6.8,0.0,24.4,0.0,8.9,32.2,25236865,35674007,17909054,12359,2807766,2745,739.02,2043.6,6001,Alameda County,CA,1671329


In [171]:
CA_landarea.shape

(12354, 71)

In [172]:
CA_landarea['population_density']= CA_landarea['population']/CA_landarea['LND110210']
CA_landarea = CA_landarea[['Date','fips','New cases','County Name','State','mask_rule_active','mask_wearing_percent',  'population','LND110210','population_density']].copy()
CA_landarea.tail(3)

,Date,fips,New cases,County Name,State,mask_rule_active,mask_wearing_percent,population,LND110210,population_density
12351,2020-08-19,6115,35,Yuba County,CA,1,0.669,78668,631.84,124.506204
12352,2020-08-20,6115,0,Yuba County,CA,1,0.669,78668,631.84,124.506204
12353,2020-08-21,6115,8,Yuba County,CA,1,0.669,78668,631.84,124.506204


## Combining mobility data 

In [173]:
# Mobility 
CA_mobility_data.head(2)

,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking
0,California,Total,2020-02-15,1.0,1.0,19.0,1.0,-1.0,0.0,30.03,1.90,58.40
1,California,Total,2020-02-16,5.0,0.0,31.0,1.0,-1.0,-1.0,6.16,-11.45,25.46


In [174]:
CA_mobility_combined = pd.merge(CA_landarea,CA_mobility_data,how='inner',left_on=['Date','County Name'],right_on=['date','county'])
CA_mobility_combined.head(2)

,Date,fips,New cases,County Name,State,mask_rule_active,mask_wearing_percent,population,LND110210,population_density,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking
0,2020-01-22,6001,0,Alameda County,CA,0,0.0,1671329,739.02,2261.547725,California,Alameda County,2020-01-22,0.0,0.0,0.0,0.0,0.0,0.0,10.27,25.12,50.04
1,2020-01-23,6001,0,Alameda County,CA,0,0.0,1671329,739.02,2261.547725,California,Alameda County,2020-01-23,0.0,0.0,0.0,0.0,0.0,0.0,8.72,20.07,36.32


In [175]:
CA_mobility_combined.shape

(11742, 22)

## Combining social distancing 

In [176]:
newyork_socialdistancing_data.head(2)

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
31068,2020-03-01,US,2,California,Alameda County,6001.0,16737,3.684,58
31069,2020-03-02,US,2,California,Alameda County,6001.0,17269,6.182,98


In [177]:
newyork_socialdistancing_data['fips'] = newyork_socialdistancing_data['fips'].apply(lambda x:int(x))
newyork_socialdistancing_data.tail(2)

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
41148,2020-09-05,US,2,California,Yuba County,6115,1284,4.008,42
41149,2020-09-06,US,2,California,Yuba County,6115,1239,2.265,24


In [178]:
CA_mobility_combined.shape

(11742, 22)

In [179]:
CA_socialDistancing_combined = pd.merge(CA_mobility_combined,newyork_socialdistancing_data,how='inner',left_on=['Date','County Name'],right_on=['date','admin2'])
CA_socialDistancing_combined.head(2)

,Date,fips_x,New cases,County Name,State,mask_rule_active,mask_wearing_percent,population,LND110210,population_density,state,county,date_x,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking,date_y,country_code,admin_level,admin1,admin2,fips_y,samples,m50,m50_index
0,2020-03-01,6001,0,Alameda County,CA,0,0.0,1671329,739.02,2261.547725,California,Alameda County,2020-03-01,6.0,9.0,15.0,1.0,4.0,0.0,-7.65,-2.79,11.07,2020-03-01,US,2,California,Alameda County,6001,16737,3.684,58
1,2020-03-02,6001,0,Alameda County,CA,0,0.0,1671329,739.02,2261.547725,California,Alameda County,2020-03-02,4.0,7.0,7.0,-1.0,3.0,0.0,6.76,6.53,13.48,2020-03-02,US,2,California,Alameda County,6001,17269,6.182,98


In [180]:
CA_socialDistancing_combined.shape

(9196, 31)

## Filtering columns for feature selection

In [181]:
columns = ['Date','fips_x','County Name','retail and recreation','grocery and pharmacy','parks','transit stations','workplaces','residential','driving','m50','m50_index','population_density','mask_rule_active','mask_wearing_percent','New cases']
CA_All_combined = CA_socialDistancing_combined[columns]

CA_All_combined.head(2)


,Date,fips_x,County Name,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,m50,m50_index,population_density,mask_rule_active,mask_wearing_percent,New cases
0,2020-03-01,6001,Alameda County,6.0,9.0,15.0,1.0,4.0,0.0,-7.65,3.684,58,2261.547725,0,0.0,0
1,2020-03-02,6001,Alameda County,4.0,7.0,7.0,-1.0,3.0,0.0,6.76,6.182,98,2261.547725,0,0.0,0


In [182]:
CA_All_combined.shape

(9196, 16)

In [183]:
CA_All_combined.to_csv('CA_combined_Sep09.csv',index=False)


# 4. Merging Social Distancing Inertia (Maryland data) and Covid data integration

In [184]:
CA_covidcases.dtypes

Date                object
Province_State      object
FIPS                 int64
Confirmed            int64
Deaths               int64
New cases            int64
New deaths           int64
mask_rule_active     int64
dtype: object

In [185]:
print(maryland_sd_inertia.shape)
CA_covidcases.shape

(732086, 45)


(12567, 8)

In [186]:
CA_socialDistancingInertiaData = pd.merge(CA_covidcases,maryland_sd_inertia,how='inner',left_on=['FIPS','Date'],right_on=['CTFIPS','date'])
print(CA_socialDistancingInertiaData.shape)

CA_socialDistancingInertiaData.head(2)

(12296, 53)


,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,CTFIPS,CTNAME,STFIPS,Social distancing index,% staying home,Trips/person,% out-of-county trips,% out-of-state trips,Miles/person,Work trips/person,Non-work trips/person,New COVID cases,Population,% change in consumption,date,Transit mode share,% people older than 60,Median income,% African Americans,% Hispanic Americans,% Male,Population density,Employment density,# hot spots/1000 people,Hospital beds/1000 people,ICUs/1000 people,# contact tracing workers/1000 people,# contact tracing workers/1000 people rank,% people older than 60 rank,COVID exposure/1000 people,#days: decreasing ILI cases,Unemployment claims/1000 people,Unemployment rate,% working from home,Cumulative inflation rate,COVID death rate,New cases/1000 people,Active cases/1000 people,#days: decreasing COVID cases,% hospital bed utilization,Testing capacity,Tests done/1000 people,% ICU utilization,Ventilator shortage,Imported COVID cases
0,2020-01-22,California,6001,0,0,0,0,0,6001,Alameda County,6,18,20,3.29,29.4,0.3,35.9,0.55,2.74,0,1666753,-2.9,2020-01-22,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0
1,2020-01-23,California,6001,0,0,0,0,0,6001,Alameda County,6,15,19,3.36,29.6,0.3,35.4,0.56,2.80,0,1666753,-0.8,2020-01-23,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0


In [210]:
filename = 'CA_socialDistancingInertiaData_'+datetime.now().strftime("%m%d") #%Y%m%d
print(filename)
CA_socialDistancingInertiaData.to_csv(filename,index=False)

CA_socialDistancingInertiaData_0909


# Feature Selection - Weightage

### Gini index

In [188]:
CA_All_combined = CA_All_combined.fillna(0)

In [189]:
def gini(list_of_values):
    sorted_list = sorted(list_of_values)
    height, area = 0, 0
    for value in sorted_list:
        height += value
        area += height - value / 2.
    fair_area = height * len(list_of_values) / 2.
    return (fair_area - area) / fair_area

In [190]:
# Calling gini-index method

feature_cols=['fips_x','retail and recreation','grocery and pharmacy',
              'parks','transit stations','workplaces','residential','driving',
              'm50','m50_index','population_density','mask_rule_active','mask_wearing_percent']
gini_values = {}
for col in feature_cols:
  dataset = CA_All_combined[col].values.tolist()
  #print(dataset)
  value = gini(dataset)
  print('cols ',col,' - gini coeff is :',value)
  gini_values[col] = value

cols  fips_x  - gini coeff is : 0.003152233209286285
cols  retail and recreation  - gini coeff is : -0.46070956381003225
cols  grocery and pharmacy  - gini coeff is : -4.70916174206947
cols  parks  - gini coeff is : 2.1143582321482133
cols  transit stations  - gini coeff is : -0.71336815224237
cols  workplaces  - gini coeff is : -0.29166458621978936
cols  residential  - gini coeff is : 0.4809245070774755
cols  driving  - gini coeff is : 1.3825737795459254
cols  m50  - gini coeff is : 0.4523632259684683
cols  m50_index  - gini coeff is : 0.4585516535107493
cols  population_density  - gini coeff is : 0.8173722127580331
cols  mask_rule_active  - gini coeff is : 0.6201609395389299
cols  mask_wearing_percent  - gini coeff is : 0.6469058351537162


In [191]:
sorted_x = sorted(gini_values.items(), key=lambda kv: kv[1])
sorted_x

[('grocery and pharmacy', -4.70916174206947),
 ('transit stations', -0.71336815224237),
 ('retail and recreation', -0.46070956381003225),
 ('workplaces', -0.29166458621978936),
 ('fips_x', 0.003152233209286285),
 ('m50', 0.4523632259684683),
 ('m50_index', 0.4585516535107493),
 ('residential', 0.4809245070774755),
 ('mask_rule_active', 0.6201609395389299),
 ('mask_wearing_percent', 0.6469058351537162),
 ('population_density', 0.8173722127580331),
 ('driving', 1.3825737795459254),
 ('parks', 2.1143582321482133)]

## Random Forest - Feature selection

In [192]:
CA_All_combined.describe().T

,count,mean,std,min,25%,50%,75%,max
fips_x,9196.0,6057.776425,33.091592,6001.000000,6029.000000,6059.000000,6085.000000,6115.000000
retail and recreation,9196.0,-25.180187,20.580526,-100.000000,-40.000000,-26.000000,-12.000000,63.000000
grocery and pharmacy,9196.0,-1.627120,14.031660,-67.000000,-10.000000,-1.000000,6.000000,86.000000
parks,9196.0,10.256851,43.714494,-83.000000,-10.000000,0.000000,25.000000,323.000000
transit stations,9196.0,-19.958895,25.598877,-86.000000,-39.000000,-17.000000,0.000000,94.000000
workplaces,9196.0,-30.944976,16.192937,-83.000000,-41.000000,-33.000000,-21.000000,14.000000
residential,9196.0,9.170400,7.903524,-3.000000,1.000000,9.000000,14.000000,37.000000
driving,9196.0,24.719081,68.432443,-80.180000,-20.520000,11.665000,50.160000,508.090000
m50,9196.0,3.053199,4.373840,0.019000,1.247750,2.450500,3.933250,322.382000
m50_index,9196.0,50.327207,158.806691,0.000000,20.000000,41.000000,64.000000,14041.000000


In [193]:
X_cols = ['fips_x','retail and recreation','grocery and pharmacy','parks',
          'transit stations','workplaces','residential','driving','m50','m50_index',
          'population_density','mask_rule_active','mask_wearing_percent']
Y_cols = ['New cases']

In [194]:
x_train_c = CA_All_combined[X_cols]
y_train_c = CA_All_combined[Y_cols]

In [195]:
x_train_c

,fips_x,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,m50,m50_index,population_density,mask_rule_active,mask_wearing_percent
0,6001,6.0,9.0,15.0,1.0,4.0,0.0,-7.65,3.684,58,2261.547725,0,0.000
1,6001,4.0,7.0,7.0,-1.0,3.0,0.0,6.76,6.182,98,2261.547725,0,0.000
2,6001,5.0,12.0,41.0,0.0,1.0,-1.0,13.19,6.271,100,2261.547725,0,0.000
3,6001,1.0,6.0,23.0,-3.0,1.0,0.0,10.67,6.209,99,2261.547725,0,0.000
4,6001,1.0,6.0,12.0,-5.0,-1.0,1.0,11.64,6.654,106,2261.547725,0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9191,6115,-11.0,12.0,0.0,-11.0,-34.0,7.0,64.29,4.100,43,124.506204,1,0.669
9192,6115,-13.0,9.0,0.0,-21.0,-32.0,8.0,54.39,4.944,52,124.506204,1,0.669
9193,6115,-13.0,5.0,0.0,-22.0,-33.0,8.0,61.72,4.090,43,124.506204,1,0.669
9194,6115,-13.0,11.0,0.0,-19.0,-33.0,8.0,59.48,4.096,43,124.506204,1,0.669


In [196]:
from sklearn.ensemble import RandomForestClassifier

# create the random forest with your hyperparameters.
rfc = RandomForestClassifier(n_estimators=150)

# fit the model to start training.
rfc.fit(x_train_c, y_train_c)

# get the importance of the resulting features.
f_importances = rfc.feature_importances_

# create a data frame for visualization.
final_df = pd.DataFrame({"Features": x_train_c.columns, "Importances":f_importances})

final_df.sort_values('Importances', ascending=False)

,Features,Importances
7,driving,0.116066
8,m50,0.109511
1,retail and recreation,0.096351
9,m50_index,0.094383
4,transit stations,0.092670
5,workplaces,0.092033
3,parks,0.091616
2,grocery and pharmacy,0.091159
6,residential,0.082193
10,population_density,0.061747


## Decision tree- feature selection

In [197]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel

clf = DecisionTreeClassifier()
clf = clf.fit(x_train_c, y_train_c)
f=clf.feature_importances_

final_dfs = pd.DataFrame({"Features": x_train_c.columns, "Importances":f})

final_dfs.sort_values('Importances', ascending=False)

,Features,Importances
7,driving,0.129130
6,residential,0.109849
8,m50,0.107853
3,parks,0.103280
2,grocery and pharmacy,0.094948
1,retail and recreation,0.093079
5,workplaces,0.087847
4,transit stations,0.082938
9,m50_index,0.078658
10,population_density,0.047219


## AutoML Implementation

In [198]:
X_cols = ['fips_x','retail and recreation','grocery and pharmacy','parks',
          'transit stations','workplaces','residential','driving','m50','m50_index',
          'population_density','mask_rule_active','mask_wearing_percent']


In [199]:
CA_All_combined['Date'] =  pd.to_datetime(CA_All_combined['Date'])

In [200]:
CA_All_combined.dtypes

Date                     datetime64[ns]
fips_x                            int64
County Name                      object
retail and recreation           float64
grocery and pharmacy            float64
parks                           float64
transit stations                float64
workplaces                      float64
residential                     float64
driving                         float64
m50                             float64
m50_index                         int64
population_density              float64
mask_rule_active                  int64
mask_wearing_percent            float64
New cases                         int64
dtype: object

In [201]:
X = CA_All_combined[X_cols]
Y = CA_All_combined[['New cases']]
Y.shape

(9196, 1)

In [202]:
X_train,X_test,Y_train, Y_test = train_test_split(X, Y,test_size=0.2, random_state=1)

In [203]:
num_folds = 12
seed = 7
scoring = 'accuracy'
models = []
models.append(('LR' , LogisticRegression()))
models.append(('LDA' , LinearDiscriminantAnalysis()))
models.append(('KNN' , KNeighborsClassifier()))
models.append(('CART' , DecisionTreeClassifier()))
models.append(('NB' , GaussianNB()))
models.append(('SVM' , SVC()))
models.append(('RF' , RandomForestClassifier(n_estimators=50)))
models.append(('XGBoost', XGBClassifier()))

In [204]:
Y_train.shape

(7356, 1)

In [118]:
results = []
names = []

for name, model in models:
    clf = model
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    accu_score = accuracy_score(Y_test, Y_pred)
    print(name + ": " + str(accu_score))

LR: 0.3972826086956522
LDA: 0.3706521739130435
KNN: 0.3690217391304348
CART: 0.296195652173913
NB: 0.23369565217391305
SVM: 0.39891304347826084
RF: 0.36630434782608695
XGBoost: 0.35978260869565215


# Reference 

* https://medium.com/next-gen-machine-learning/feature-selection-best-methods-for-feature-selection-python-f3536aad5b4a